In [2]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

In [3]:
import geopandas as gpd
from shapely import wkt

# Pull in geojson address data from openaddresses
fname = "../../data/addresses/us_il_city_of_chicago-addresses-state.geojson"

# read file to dataframe
gdf = gpd.read_file(fname)

# Verify/Make pandas dataframe
addressdf = pd.DataFrame(gdf)

# Change 'geometry' column datatype from geometry to object
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x))

In [4]:
# Check dataframe
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,7726,W JARLATH ST,,,,,2cac3b0c78b34539,POINT (-87.8200365999999946 42.0125603999999981)
1,,,7724,W JARLATH ST,,,,,08fc96e7f8cefc4d,POINT (-87.8198816000000022 42.0125639000000035)
2,,,7720,W JARLATH ST,,,,,e3ad1acd7a000511,POINT (-87.8197057999999942 42.0125677999999994)
3,,,7714,W JARLATH ST,,,,,d63788ef9552e1da,POINT (-87.8195391999999941 42.0125715000000000)
4,,,7248,N OVERHILL AVE,,,,,5f66e762c70f7374,POINT (-87.8191652999999945 42.0128339999999980)
...,...,...,...,...,...,...,...,...,...,...
605655,,,1870,N DAMEN AVE,,,,,9c880cac52796c98,POINT (-87.6778917000000035 41.9158554999999993)
605656,,,5014,S KING DR,,,,,72b4137c242c38f7,POINT (-87.6170838000000032 41.8033525000000026)
605657,,,2236,W BELMONT AVE,,,,,672db80a65ce1a2b,POINT (-87.6847611000000029 41.9397601000000009)
605658,,,2318,S CANAL ST,,,,,81dc4df792504c55,POINT (-87.6388280999999978 41.8502095999999995)


In [5]:
# Combine address columns to one column
addressdf['addresses'] = addressdf['number'].str.cat(addressdf['street'] + ", Chicago, Illinois",sep=" ")

# Remove extra strings from the latitude and longitude in the geometry column
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry,addresses
0,,,7726,W JARLATH ST,,,,,2cac3b0c78b34539,-87.8200365999999946 42.0125603999999981,"7726 W JARLATH ST, Chicago, Illinois"
1,,,7724,W JARLATH ST,,,,,08fc96e7f8cefc4d,-87.8198816000000022 42.0125639000000035,"7724 W JARLATH ST, Chicago, Illinois"
2,,,7720,W JARLATH ST,,,,,e3ad1acd7a000511,-87.8197057999999942 42.0125677999999994,"7720 W JARLATH ST, Chicago, Illinois"
3,,,7714,W JARLATH ST,,,,,d63788ef9552e1da,-87.8195391999999941 42.0125715000000000,"7714 W JARLATH ST, Chicago, Illinois"
4,,,7248,N OVERHILL AVE,,,,,5f66e762c70f7374,-87.8191652999999945 42.0128339999999980,"7248 N OVERHILL AVE, Chicago, Illinois"
...,...,...,...,...,...,...,...,...,...,...,...
605655,,,1870,N DAMEN AVE,,,,,9c880cac52796c98,-87.6778917000000035 41.9158554999999993,"1870 N DAMEN AVE, Chicago, Illinois"
605656,,,5014,S KING DR,,,,,72b4137c242c38f7,-87.6170838000000032 41.8033525000000026,"5014 S KING DR, Chicago, Illinois"
605657,,,2236,W BELMONT AVE,,,,,672db80a65ce1a2b,-87.6847611000000029 41.9397601000000009,"2236 W BELMONT AVE, Chicago, Illinois"
605658,,,2318,S CANAL ST,,,,,81dc4df792504c55,-87.6388280999999978 41.8502095999999995,"2318 S CANAL ST, Chicago, Illinois"


In [6]:
# Remove un-needed columns
addressdf.drop(['unit', 'number', 'street', 'region', 'city', 'district', 'hash'],axis=1,inplace=True)
addressdf

,id,postcode,geometry,addresses
0,,,-87.8200365999999946 42.0125603999999981,"7726 W JARLATH ST, Chicago, Illinois"
1,,,-87.8198816000000022 42.0125639000000035,"7724 W JARLATH ST, Chicago, Illinois"
2,,,-87.8197057999999942 42.0125677999999994,"7720 W JARLATH ST, Chicago, Illinois"
3,,,-87.8195391999999941 42.0125715000000000,"7714 W JARLATH ST, Chicago, Illinois"
4,,,-87.8191652999999945 42.0128339999999980,"7248 N OVERHILL AVE, Chicago, Illinois"
...,...,...,...,...
605655,,,-87.6778917000000035 41.9158554999999993,"1870 N DAMEN AVE, Chicago, Illinois"
605656,,,-87.6170838000000032 41.8033525000000026,"5014 S KING DR, Chicago, Illinois"
605657,,,-87.6847611000000029 41.9397601000000009,"2236 W BELMONT AVE, Chicago, Illinois"
605658,,,-87.6388280999999978 41.8502095999999995,"2318 S CANAL ST, Chicago, Illinois"


In [10]:

fltraddress = addressdf[addressdf['addresses'].str.lower().str.contains('n drake')]
fltraddress

,id,postcode,geometry,addresses
34273,,,-87.7181443000000058 41.9953179000000034,"6252 N DRAKE AVE, Chicago, Illinois"
34274,,,-87.7181346999999931 41.9949871999999971,"6240 N DRAKE AVE, Chicago, Illinois"
34275,,,-87.7181322999999935 41.9949045999999981,"6236 N DRAKE AVE, Chicago, Illinois"
34276,,,-87.7181298999999939 41.9948218999999980,"6234 N DRAKE AVE, Chicago, Illinois"
34277,,,-87.7181274999999943 41.9947391999999979,"6230 N DRAKE AVE, Chicago, Illinois"
...,...,...,...,...
602524,,,-87.7163558000000023 41.9389757000000003,"3148 N DRAKE AVE, Chicago, Illinois"
602949,,,-87.7153265000000033 41.9157748000000012,"1904 N DRAKE AVE, Chicago, Illinois"
603251,,,-87.7153088999999966 41.9152009000000021,"1844 N DRAKE AVE, Chicago, Illinois"
603873,,,-87.7164120000000054 41.9594108000000006,"4301 N DRAKE AVE, Chicago, Illinois"


In [11]:
adds_sml = fltraddress.sample(n = 100) 
adds_sml

,id,postcode,geometry,addresses
102309,,,-87.7165889999999990 41.9408688999999981,"3254 N DRAKE AVE, Chicago, Illinois"
114123,,,-87.7154828000000037 41.9267892000000018,"2509 N DRAKE AVE, Chicago, Illinois"
201539,,,-87.7155269000000004 41.8978271000000007,"914 N DRAKE AVE, Chicago, Illinois"
36061,,,-87.7171555999999981 41.9855730000000023,"5725 N DRAKE AVE, Chicago, Illinois"
144599,,,-87.7145641000000040 41.9124432999999996,"1719 N DRAKE AVE, Chicago, Illinois"
...,...,...,...,...
103452,,,-87.7149854000000033 41.9396112000000016,"3209 N DRAKE AVE, Chicago, Illinois"
100492,,,-87.7161248000000029 41.9495542000000015,"3741 N DRAKE AVE, Chicago, Illinois"
208729,,,-87.7145371000000011 41.8883873999999992,"405 N DRAKE AVE, Chicago, Illinois"
36416,,,-87.7171084000000008 41.9839820999999986,"5633 N DRAKE AVE, Chicago, Illinois"


In [13]:
#adds_sml.to_csv (r'static/data/sample/adds_sml.csv', index = True, header=True)

In [17]:
adds_sml = pd.read_csv("static/data/sample/adds_sml.csv")
adds_sml

,Unnamed: 0,id,postcode,geometry,addresses
0,102309,NaN,NaN,-87.7165889999999990 41.9408688999999981,"3254 N DRAKE AVE, Chicago, Illinois"
1,114123,NaN,NaN,-87.7154828000000037 41.9267892000000018,"2509 N DRAKE AVE, Chicago, Illinois"
2,201539,NaN,NaN,-87.7155269000000004 41.8978271000000007,"914 N DRAKE AVE, Chicago, Illinois"
3,36061,NaN,NaN,-87.7171555999999981 41.9855730000000023,"5725 N DRAKE AVE, Chicago, Illinois"
4,144599,NaN,NaN,-87.7145641000000040 41.9124432999999996,"1719 N DRAKE AVE, Chicago, Illinois"
...,...,...,...,...,...
95,103452,NaN,NaN,-87.7149854000000033 41.9396112000000016,"3209 N DRAKE AVE, Chicago, Illinois"
96,100492,NaN,NaN,-87.7161248000000029 41.9495542000000015,"3741 N DRAKE AVE, Chicago, Illinois"
97,208729,NaN,NaN,-87.7145371000000011 41.8883873999999992,"405 N DRAKE AVE, Chicago, Illinois"
98,36416,NaN,NaN,-87.7171084000000008 41.9839820999999986,"5633 N DRAKE AVE, Chicago, Illinois"


In [18]:
# '''addresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
#          "2622 N Drake Ave, Chicago, Illinois 60647",
#          "2626 N Drake Ave, Chicago, Illinois 60647",
#          "2630 N Drake Ave, Chicago, Illinois 60647",
#          "2648 N Drake Ave, Chicago, Illinois 60647"]'''

In [19]:
# '''df = DataFrame (addresses, columns=['Address'])
# df'''

In [20]:
# '''API_KEY = os.getenv("API1234")
# g = GoogleV3(api_key=gkey)


# loc_coordinates = []
# loc_address = []
# for address in df.Address:
#     try:
#         inputAddress = address
#         location = g.geocode(inputAddress, timeout=15)
#         loc_coordinates.append((location.latitude, location.longitude))
#         loc_address.append(inputAddress)
#     except Exception as e:
#         print('Error, skipping address...', e)


# df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
# df_geocodes'''

In [ ]:
import urllib.request, urllib.parse 
import requests
from requests.utils import requote_uri
import json
# Import google_streetview for the api module
import google_streetview.api


# Define parameters for street view api
params = [{
    'size': '600x300', # max 640x640 pixels
    'location': '41.9295582,-87.7159139',
    'heading': '90',
    'pitch': '-0.76',
    'key': gkey
}]

# Create a results object
results = google_streetview.api.results(params)

# Download images to directory 'downloads'
results.download_links('static/data/raw')

In [ ]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM ADDRESSES PROVIDED
'''
myloc = "static/data/raw"
key = key = "&key=" + "gkey"
def GetStreet(Add,SaveLoc):
    base = "https://maps.googleapis.com/maps/api/streetview?size=1200x800&location="
    #MyUrl = base + urllib.parse.quote_plus(Add) + key  #added url encoding
    MyUrl = base + requote_uri(Add) + key  #added url encoding
    fi = Add + ".jpg"
    requests.get(MyUrl, os.path.join(SaveLoc,fi), stream=True, headers={'User-agent': 'Mozilla/5.0'})
  

Tests = ["2620 N Drake Ave, Chicago, Illinois 60647",
         "2622 N Drake Ave, Chicago, Illinois 60647",
         "2626 N Drake Ave, Chicago, Illinois 60647",
         "2630 N Drake Ave, Chicago, Illinois 60647",
         "2648 N Drake Ave, Chicago, Illinois 60647"]

for i in Tests:
 GetStreet(Add=i,SaveLoc=myloc)
''' 
  
    

In [ ]:

'''
# load and show an image with Pillow

# load the image
image = Image.open('static/data/test/brickornot.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()'''

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/test/brickornot.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/brickornot.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/brickornot.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''